# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import seaborn as sns

# Import API key
from api_keys import g_key






### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:

#source: in class examples
# Study data files
pwd = os.path.abspath('.')
city_weather_path = os.path.join(pwd, 'cityweather.csv')
city_weather = pd.read_csv(city_weather_path)
city_weather

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,Cape Town,-33.9258,18.4232,291.08,85,0,0.45,ZA,1633988204
1,Dikson,73.5069,80.5464,272.48,67,100,6.38,RU,1633988455
2,Vardø,70.3705,31.1107,279.26,96,100,2.93,NO,1633988455
3,Albany,42.6001,-73.9662,294.38,77,25,0.79,US,1633988394
4,Hobart,-42.8794,147.3294,282.66,81,40,0.45,AU,1633988456
...,...,...,...,...,...,...,...,...,...
575,Pandan,14.0453,124.1694,300.52,83,100,7.58,PH,1633988644
576,Ambon City,-3.6954,128.1814,297.23,94,20,2.06,ID,1633988645
577,Manokwari,-0.8667,134.0833,300.15,74,59,1.69,ID,1633988636
578,Muzhi,65.3667,64.6667,269.15,97,73,1.58,RU,1633988645


In [16]:
#Create humidity heatmap

In [19]:
#create heat map
#Source: In class activity
gmaps.configure(api_key=g_key)



locations = city_weather[["lat", "lng"]].astype(float)
# city_weather = city_weather.dropna()
humidity_rate = city_weather['humidity'].astype(float)
humidity_rate 




fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=max(city_weather['humidity']),
                                 point_radius = 2)

fig.add_layer(heat_layer)


fig


Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#create heat map
#Source: In class activity, Day 3 Activity 10

gmaps.configure(api_key=g_key)



locations = city_weather[["lat", "lng"]].astype(float)
# city_weather = city_weather.dropna()
humidity_rate = city_weather['humidity'].astype(float)
humidity_rate 




fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=max(city_weather['humidity']),
                                 point_radius = 2)

fig.add_layer(heat_layer)


fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#Find ideal weather

ideal_weather = city_weather[(city_weather.humidity <= 30) & (city_weather.cloudiness == 0) & (city_weather.wind_speed >= 2.5)]

ideal_weather

#no null values to drop

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
68,Mar del Plata,-38.0023,-57.5575,288.16,30,0,4.02,AR,1633988479
204,Kharan,28.5833,65.4167,289.04,13,0,4.12,PK,1633988524
261,Calama,-22.4667,-68.9333,299.42,6,0,9.26,CL,1633988543
275,Eenhana,-17.4667,16.3333,302.01,25,0,3.73,NaN,1633988548
477,Turbat,26.0023,63.0440,297.99,9,0,8.61,PK,1633988517
504,Comodoro Rivadavia,-45.8667,-67.5000,295.07,21,0,8.75,AR,1633988622


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
#Store into variable named hotel_df

hotel_df = ideal_weather
hotel_df

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
68,Mar del Plata,-38.0023,-57.5575,288.16,30,0,4.02,AR,1633988479
204,Kharan,28.5833,65.4167,289.04,13,0,4.12,PK,1633988524
261,Calama,-22.4667,-68.9333,299.42,6,0,9.26,CL,1633988543
275,Eenhana,-17.4667,16.3333,302.01,25,0,3.73,NaN,1633988548
477,Turbat,26.0023,63.0440,297.99,9,0,8.61,PK,1633988517
504,Comodoro Rivadavia,-45.8667,-67.5000,295.07,21,0,8.75,AR,1633988622


In [37]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# source: In class activity Day 3, Activity 4 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng from df
    lat = row['lat']
    lng = row['lng']

    # add location to params dict
    params['location'] = f"{lat}, {lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Nearest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 68: Mar del Plata.
Nearest hotel is Gran Hotel Mar del Plata.
------------
Retrieving Results for Index 204: Kharan.


C:\Users\mom\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\mom\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Nearest hotel is Dil Aram Bangla.
------------
Retrieving Results for Index 261: Calama.
Nearest hotel is Diego De Almagro Calama.
------------
Retrieving Results for Index 275: Eenhana.
Nearest hotel is Hotel Litu.
------------
Retrieving Results for Index 477: Turbat.
Nearest hotel is Mubashar Quarters.
------------
Retrieving Results for Index 504: Comodoro Rivadavia.
Nearest hotel is Austral Hotel.
------------


In [38]:
hotel_df

,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date,hotel_name,Hotel Name
68,Mar del Plata,-38.0023,-57.5575,288.16,30,0,4.02,AR,1633988479,Gran Hotel Mar del Plata,Gran Hotel Mar del Plata
204,Kharan,28.5833,65.4167,289.04,13,0,4.12,PK,1633988524,Dil Aram Bangla,Dil Aram Bangla
261,Calama,-22.4667,-68.9333,299.42,6,0,9.26,CL,1633988543,Diego De Almagro Calama,Diego De Almagro Calama
275,Eenhana,-17.4667,16.3333,302.01,25,0,3.73,NaN,1633988548,Hotel Litu,Hotel Litu
477,Turbat,26.0023,63.0440,297.99,9,0,8.61,PK,1633988517,Mubashar Quarters,Mubashar Quarters
504,Comodoro Rivadavia,-45.8667,-67.5000,295.07,21,0,8.75,AR,1633988622,Austral Hotel,Austral Hotel


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [47]:
# Plot markers on top of the heatmap.
#Source In Class Activity Day 3
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

nearest_hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


fig = gmaps.figure()
fig.add_layer(nearest_hotel_layer)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))